In [3]:
#Samagra Parishrama Survey 2020 Department of Industries

import requests
from bs4 import BeautifulSoup

url = 'https://www.apindustries.gov.in/SPS/UserAccount/LocationMap.aspx'
response = requests.get(url)
print(response) #checking if it worked ; 200- Okay

soup = BeautifulSoup(response.text, 'html.parser') #parsing
mydiv = soup.find("div", attrs={"class": "container-fluid"})  #finding the parent div tag
scripts = mydiv.findAll('script')[1] #getting the script tag
scripts_Str = scripts.string.strip()
scripts_Str = scripts_Str.replace('\r\n','')
scripts_Str=scripts_Str[:-1]+" "+scripts_Str[-1]
scripts_Str[-20:]


<Response [200]>


'                }] ;'

In [4]:
import re
# import json

pattern = 'var markers = (.*?) ;'
p = re.match(pattern, scripts_Str)
dataform = str(p.groups()[0]).strip("'<>() ").replace('\'', '\"')
# dataform[:10000]
dataform=dataform.split("}    ,")   #splitting companies by },
# print(len(dataform)) - 36271
dataform=[i.split("            {            ")[1].strip() for i in dataform]  #cleaning companies by also removing { by making into a list
# print(len(dataform)) - 36271
dataform[:5]


['"title": "Sri padamavathi mechanical works",            "lat": "16.4971132",            "lng": "80.6779712",           // "description": "Krishna",           "description": "<table><tr style="height:30px;"><td style="width:180px;">Industry Name</td><td style="width:20px;">:</td><td><b>"+"Sri padamavathi mechanical works"+"</b></td></tr><tr style="height:30px;"><td>District Name</td><td>:</td><td><b>"+"Krishna"+"</b></td></tr><tr style="height:30px;"><td>Sector Name</td><td>:</td><td><b>"+"SERVICE"+"</b></td></tr><tr style="height:30px;"><td>Activity Name</td><td>:</td><td><b>"+"REPAIR AND MAINTENANCE OF BUSES"+"</b></td></tr><tr style="height:30px;"><td>Industry as per pollution Index Category</td><td>:</td><td><b>"+"White"+"</b></td></tr><tr style="height:30px;"><td>Total Workers</td><td>:</td><td><b>"+"1"+"</b></td></tr></table><b>",       "category":"Micro.png"',
 '"title": "venkatesh mechanical works",            "lat": "13.2185764",            "lng": "79.1249472",           // "

In [5]:
x1 = """<table><tr style="height:30px;"><td style="width:180px;">"""
x2 = """</td><td style="width:20px;">:</td><td><b>"+"""
x3 = """+"</b></td></tr><tr style="height:30px;"><td>"""
x4 = """</td><td>:</td><td><b>"+"""
x5 = """+"</b></td></tr></table><b>"""
dataform=[i.replace(x1," ").replace(x2," ").replace(x3, " ").replace(x4, " ").replace(x5," ") for i in dataform]  #removing the tags in description to clean it
# dataform[:5]

dataform1 = [[j.strip() for j in i.split(",")]for i in dataform]  #splitting a company by type
len(dataform1[0])
dataform1[:30]

[['"title": "Sri padamavathi mechanical works"',
  '"lat": "16.4971132"',
  '"lng": "80.6779712"',
  '// "description": "Krishna"',
  '"description": " Industry Name "Sri padamavathi mechanical works" District Name "Krishna" Sector Name "SERVICE" Activity Name "REPAIR AND MAINTENANCE OF BUSES" Industry as per pollution Index Category "White" Total Workers "1" "',
  '"category":"Micro.png"'],
 ['"title": "venkatesh mechanical works"',
  '"lat": "13.2185764"',
  '"lng": "79.1249472"',
  '// "description": "Chittoor"',
  '"description": " Industry Name "venkatesh mechanical works" District Name "Chittoor" Sector Name "SERVICE" Activity Name "REPAIR AND MAINTENANCE OF BUSES" Industry as per pollution Index Category "Green" Total Workers "1" "',
  '"category":"Micro.png"'],
 ['"title": "N Nagaraju Transformers"',
  '"lat": "14.4516162"',
  '"lng": "78.8407888"',
  '// "description": "Kadapa"',
  '"description": " Industry Name "N Nagaraju Transformers" District Name "Kadapa" Sector Name "SE

In [6]:
import pandas as pd
df = pd.DataFrame(columns=['Industry Name','Latitude','Longitude','Category', 'District Name','Sector Name','Activity Name','Pollution Index Category','Total Workers'])
for company in dataform1:
#   print(company)
  IN = company[0][10:len(company[0])-1]
  LT = company[1][8:len(company[1])-1]
  LN = company[2][8:len(company[2])-1]
  CT = company[-1][12:len(company[-1])-5]
  des = list(filter(("").__ne__,company[4].split("\"")[3:-2]))
  print(des)
  DN = des[3]
  if len(des)== 4:
      des.append('NA')
      des.append('NA')
  SN = des[5]
  if len(company)>6:
      a = list(filter(("").__ne__,company[5].split("\"")[0:1]))
      b = list(filter(("").__ne__,company[5].split("\"")[2:3]))
      if len(b) == 0:
          b.append('NA')
      c = list(filter(("").__ne__,company[5].split("\"")[3:-2]))
      if len(c) < 2:
          c.append('NA')
          c.append('NA')
    #   print(c)
      AN = a[0]
      PI = b[0]
      TW = c[1]
  else:
    AN = des[7]
    PI = des[9]
    if len(des) == 11:
        des.append('NA')
    TW = des[11]
  df = df.append({'Industry Name' : IN, 'Latitude' : LT, 'Longitude' : LN,'District Name' :DN,'Sector Name':SN,'Activity Name':AN,'Pollution Index Category':PI,'Category':CT,'Total Workers':TW},  
                ignore_index = True)
df['Total Workers'].unique()

Streaming output truncated to the last 5000 lines.
[' Industry Name ', 'sree umamaheswara silpakala mandiramu', ' District Name ', 'Kurnool', ' Sector Name ', 'Non Mettalic Mineral', ' Activity Name ', 'stone carving', ' Industry as per pollution Index Category ', 'Green', ' Total Workers ', '2']
[' Industry Name ', 'vani Balasubramanyam silpa kala mandiram', ' District Name ', 'Kurnool', ' Sector Name ', 'Non Mettalic Mineral', ' Activity Name ', 'stone carving', ' Industry as per pollution Index Category ', 'Green', ' Total Workers ', '3']
[' Industry Name ', 'SRI VANI JAGADEESHWARA SILPA KALA MANDIRAM', ' District Name ', 'Kurnool', ' Sector Name ', 'Non Mettalic Mineral', ' Activity Name ', 'stone carving', ' Industry as per pollution Index Category ', 'Green', ' Total Workers ', '2']
[' Industry Name ', 'sharada shilpa Kala mandiram', ' District Name ', 'Kurnool', ' Sector Name ', 'Non Mettalic Mineral', ' Activity Name ', 'stone carving', ' Industry as per pollution Index Categor

array(['1', '4', '2', '3', '20', '8', '5', '6', '14', 'NA', '7', '10',
       '24', '15', '22', '19', '12', '61', '9', '81', '28', '13', '35',
       '27', '21', '11', '26', '16', '29', '17', '36', '40', '711', '670',
       '141', '95', '46', '31', '50', '18', '25', '902', '147', '103',
       '138', '49', '91', '72', '113', '76', '400', '105', '360', '170',
       '185', '55', '101', '430', '500', '389', '162', '51', '30', '228',
       '480', '123', '130', '131', '88', '142', '420', '96', '73', '62',
       '42', '63', '187', '106', '80', '94', '45', '109', '43', '69',
       '52', '32', '322', '321', '47', '271', '217', '407', '41', '111',
       '39', '334', '33', '23', '166', '38', '34', '315', '412', '252',
       '165', '155', '137', '139', '251', '204', '146', '53', '725',
       '275', '286', '86', '1000', '153', '213', '44', '847', '260',
       '143', '642', '349', '37', '60', '240', '200', '250', '92', '70',
       '175', '119', '122', '74', '100', '75', '152', '136', '234

In [7]:
df.shape

(36252, 9)

In [8]:
df

,Industry Name,Latitude,Longitude,Category,District Name,Sector Name,Activity Name,Pollution Index Category,Total Workers
0,Sri padamavathi mechanical works,16.4971132,80.6779712,Micro,Krishna,SERVICE,REPAIR AND MAINTENANCE OF BUSES,White,1
1,venkatesh mechanical works,13.2185764,79.1249472,Micro,Chittoor,SERVICE,REPAIR AND MAINTENANCE OF BUSES,Green,1
2,N Nagaraju Transformers,14.4516162,78.8407888,Micro,Kadapa,SERVICE,Generators and Transformers,Green,4
3,Sri Durga engineering works,16.507045,80.7044161,Micro,Krishna,SERVICE,Generators and Transformers,Green,1
4,Srinivasa Electricals,16.5043265,80.7025557,Micro,Krishna,SERVICE,Generators and Transformers,Green,1
...,...,...,...,...,...,...,...,...,...
36247,HANUMAN COIR PRODUCTS,16.4996409,82.0180639,Micro,East Godavari,COIR BASED,COCONUT FIBRE,White,7
36248,PAVAN COIR INDUSTRY,16.4996409,82.0180639,Micro,East Godavari,COIR BASED,COCONUT FIBRE,White,2
36249,Flour mill,16.3226369,80.9606974,Micro,Krishna,COIR BASED,COCONUT FIBRE,Green,1
36250,Rama Industries,16.5880314,81.7539192,Micro,West Godavari,COIR BASED,COCONUT FIBRE,Green,2


In [ ]:
df.to_csv('AP_Industries_location.csv')